In [1]:
from datetime import datetime
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

In [7]:
# database =
# user =
# password =
# host =
# port =

In [8]:
CONNECTION = f"postgresql://{user}:{password}@{host}:{port}/{database}"

In [24]:
default_posts_data = pd.read_sql(
    """
    SELECT * FROM public.post_text_df
    """,
    con=CONNECTION
).rename({'post_id': 'id'}, axis=1)

In [9]:
post_data = pd.read_sql(
    """
    SELECT * FROM public.my_post_data
    """,
    con=CONNECTION
)

In [10]:
df_time_data = pd.read_sql(
    f"""
    SELECT * FROM public.my_time_data
    ORDER BY RANDOM()
    LIMIT 100000;
    """,
    con=CONNECTION
)

In [23]:
id_ = 2024
time = datetime(2021,11,11)
limit = 5

In [12]:
df_user = pd.read_sql(
    f"""
    SELECT * FROM public.my_user_data u
    WHERE u.user_id = {id_}
    """,
    con=CONNECTION
).drop('user_id', axis=1)

In [13]:
df_user['month'] = time.month
df_user['weekday'] = time.weekday()
df_user['hour'] = time.hour
df_user['minute'] = time.minute

In [14]:
df_user = pd.concat([df_user] * len(post_data))
df_user = df_user.reset_index().drop('index', axis=1)
df_user = pd.concat((df_user, post_data), axis=1)
df_user = df_user.set_index('post_id')

In [15]:
df_user = df_user[['gender', 'age', 'country', 'city', 
                   'exp_group', 'month', 'weekday',
                   'hour', 'minute', 'topic', 'text_len', 
                   'cluster_0', 'cluster_1', 'cluster_2', 
                   'cluster_3', 'cluster_4', 'cluster_5', 
                   'cluster_6', 'os_iOS', 'source_organic'
                  ]]

In [16]:
df_user.head(2)

,gender,age,country,city,exp_group,month,weekday,hour,minute,topic,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,os_iOS,source_organic
post_id,,,,,,,,,,,,,,,,,,,,
1,-1.108519,0.762233,-0.329883,0.698471,-1.077545,11,3,0,0,-0.218296,0.520822,1.766706,-0.305010,1.228945,0.921282,1.511576,0.171627,0.79118,1.360731,-0.777821
2,-1.108519,0.762233,-0.329883,0.698471,-1.077545,11,3,0,0,-0.218296,1.078160,1.073711,-1.295447,1.001209,-0.274666,0.975423,-2.346733,0.02171,1.360731,-0.777821


In [17]:
user_time = pd.DataFrame.from_dict(
        {'index': [len(df_time_data)],
         'month': [time.month],
         'weekday': [time.weekday()],
         'hour': [time.hour],
         'minute': [time.minute]}
    )

In [18]:
df_time = pd.concat((df_time_data, user_time), axis=0)
df_time = StandardScaler().fit_transform(df_time.iloc[:, 1:])

df_user[['month', 'weekday', 'hour', 'minute']] = df_time[-1]

In [19]:
df_user.head()

,gender,age,country,city,exp_group,month,weekday,hour,minute,topic,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,os_iOS,source_organic
post_id,,,,,,,,,,,,,,,,,,,,
1,-1.108519,0.762233,-0.329883,0.698471,-1.077545,0.0237,0.011417,-2.912407,-1.708422,-0.218296,0.520822,1.766706,-0.305010,1.228945,0.921282,1.511576,0.171627,0.791180,1.360731,-0.777821
2,-1.108519,0.762233,-0.329883,0.698471,-1.077545,0.0237,0.011417,-2.912407,-1.708422,-0.218296,1.078160,1.073711,-1.295447,1.001209,-0.274666,0.975423,-2.346733,0.021710,1.360731,-0.777821
3,-1.108519,0.762233,-0.329883,0.698471,-1.077545,0.0237,0.011417,-2.912407,-1.708422,-0.218296,1.572559,0.687403,-1.579254,0.716623,-0.115016,0.602152,-0.409742,-0.106848,1.360731,-0.777821
4,-1.108519,0.762233,-0.329883,0.698471,-1.077545,0.0237,0.011417,-2.912407,-1.708422,-0.218296,-0.210426,0.086207,-2.080318,-0.099701,-0.514185,0.337472,-0.630977,0.075599,1.360731,-0.777821
5,-1.108519,0.762233,-0.329883,0.698471,-1.077545,0.0237,0.011417,-2.912407,-1.708422,-0.218296,-0.319740,-0.742106,-1.752488,-0.867388,-1.243620,-0.055057,-0.802418,-0.016528,1.360731,-0.777821


In [20]:
model = joblib.load('CB_best.pkl')

In [21]:
preds = model.predict_proba(df_user)[:,1]

In [22]:
top_of_preds = pd.DataFrame(preds, df_user.index, columns=['value']).sort_values(by='value', ascending=False)
top_of_preds.head()

,value
post_id,
6770,0.234834
6696,0.233384
5387,0.233348
7191,0.233158
5656,0.232625


In [25]:
result = top_of_preds[:limit].index.values
result

array([6770, 6696, 5387, 7191, 5656], dtype=int64)

In [27]:
all_result = []

for i in result:
    all_result.append(
        default_posts_data[default_posts_data.id == i].iloc[0, :].to_dict()
    )

In [28]:
all_result

[{'id': 6770,
  'text': 'Really bad. Why anyone thinks this is a good film let alone funny is a true mystery. I like comedies as much as the next man and I LOVED A Christmas Story. The fact that it has the same director and was based on the same writers memoirs has me completely puzzled as to why this film is such a complete failure on every level. Charles Grodin is woefully miscast as the father for starters. For another it does not seem to have the same pacing -- it just doesnt flow well. Everything seems tired and forced. The joy of life that permeated the first film is completely absent here -- you just want the movie to end. I wouldnt even recommend this movie for curiosity-seekers who enjoyed A Christmas Story. Its that bad. 1/10.',
  'topic': 'movie'},
 {'id': 6696,
  'text': 'I have seen romantic comedies and this is one of the easiest/worst attempts at one. A lot of the scenes work in a plug-and-play manner inserted strictly to conform to the romantic-comedy genre. Usually thi